In [17]:
pip install spectral

Note: you may need to restart the kernel to use updated packages.


In [18]:
### Loading Earth Lab Spectral Tools

# 1. Enable autoreload in your Jupyter Notebook:

%load_ext autoreload
%autoreload 2

# 2. Import the custom tools module:

import spectral_unmixing_tools as el_spectral

# 3. Verify that the tools loaded correctly by printing the module's directory:

print(dir(el_spectral))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['ENVIProcessor', 'GradientBoostingRegressor', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'apply_topo_and_brdf_corrections', 'boosted_quantile_plot', 'boosted_quantile_plot_by_sensor', 'box', 'clean_data_and_write_to_csv', 'concatenate_sensors', 'control_function', 'download_neon_file', 'download_neon_flight_lines', 'extract_overlapping_layers_to_2d_dataframe', 'find_raster_files', 'fit_models_with_different_alpha', 'flight_lines_to_envi', 'generate_config_json', 'generate_correction_configs', 'generate_correction_configs_for_directory', 'get_spectral_data_and_wavelengths', 'glob', 'go_forth_and_multiply', 'gpd', 'h5py', 'ht', 'jefe', 'json', 'load_and_combine_rasters', 'load_spectra', 'mask', 'np', 'os', 'pd', 'plot_each_sensor_with_highlight', 'plot_spectral_data', 'plot_with_highlighted_sensors', 'plt', 'prepare_spectral_data', 'proce

In [50]:
ray.shutdown()
import ray

import os
os.environ["RAY_memory_usage_threshold"] = "0.9"  # Allow up to 90% memory usage
import ray
ray.init(
    object_store_memory=int(300 * 1024 ** 3),
    _temp_dir="/tmp/ray"
)


ValueError: After taking into account object store and redis memory usage, the amount of memory on this node available for tasks and actors (-321.7 GB) is less than -3745% of total. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).

In [51]:
import ray

ray.init()
@ray.remote
def test_task(x):
    return x * 2

result = ray.get(test_task.remote(5))
print(f"Ray task completed successfully: {result}")


ValueError: Attempting to cap object store memory usage at 54470246 bytes, but the minimum allowed is 78643200 bytes.

In [45]:
import os
import glob
import subprocess
import ray
import ray



def apply_topo_and_brdf_corrections(base_folder_path, conda_env_path='/opt/conda/envs/macrosystems'):
    # Construct the full path to the Python executable in the specified Conda environment
    python_executable = os.path.join(conda_env_path, "bin", "python")
    print("Starting topo and BRDF correction. This takes a long time.")
    
    # Find all subfolders in the base folder
    subfolders = [f for f in glob.glob(os.path.join(base_folder_path, '*')) if os.path.isdir(f)]
    
    for folder in subfolders:
        folder_name = os.path.basename(os.path.normpath(folder))
        json_file_name = f"{folder_name}_config__envi.json"
        json_file_path = os.path.join(folder, json_file_name)  # Removed unnecessary slash
        
        print(f"Processing folder: {folder}")
        print(f"Looking for JSON file: {json_file_path}")
        
        # Check if the JSON file exists
        if os.path.isfile(json_file_path):
            # Call the script with the JSON file path
            command = f"{python_executable} image_correct.py {json_file_path}"
            process = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            
            if process.returncode != 0:
                print(f"Error executing command: {command}")
                print(f"Standard Output: {process.stdout}")
                print(f"Error Output: {process.stderr}")
            else:
                print(f"Successfully processed: {json_file_path}")
                print(f"Standard Output: {process.stdout}")
        else:
            print(f"JSON file not found: {json_file_path}")
    
    print("All done!")


ValueError: After taking into account object store and redis memory usage, the amount of memory on this node available for tasks and actors (-212.66 GB) is less than -2475% of total. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).

In [41]:
#~13 minutes per file. 5 files here for ~65 minute runtime
base_folder = "NIWOT_calibration_flight_08_2020"
apply_topo_and_brdf_corrections(base_folder)


Starting topo and BRDF correction. This takes a long time.
Processing folder: NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200807_170802_reflectance
Looking for JSON file: NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200807_170802_reflectance/NEON_D13_NIWO_DP1_20200807_170802_reflectance_config__envi.json
Error executing command: /opt/conda/envs/macrosystems/bin/python image_correct.py NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200807_170802_reflectance/NEON_D13_NIWO_DP1_20200807_170802_reflectance_config__envi.json
Standard Output: NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200807_170802_reflectance/NEON_D13_NIWO_DP1_20200807_170802_reflectance_config__envi.json
Using 1 CPUs.

Error Output: 2024-12-06 20:02:21,602	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you 

In [25]:
import psutil

def log_memory():
    memory = psutil.virtual_memory()
    print(f"Memory Usage: {memory.percent}%")

log_memory()

Memory Usage: 5.0%


In [47]:
import psutil

available_memory = psutil.virtual_memory().available
print(f"Available Memory: {available_memory / (1024 ** 3):.2f} GB")

Available Memory: 473.94 GB
